In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
df = pd.read_csv("data/triples.txt", sep= " ", header=None, names=['Artiste1', 'Lien', 'Artiste2','.'])

In [3]:
metadata = pd.read_csv('data/metadata_dataframe.csv')

In [4]:
corresp = metadata[["artist_url", 'Name']].set_index('artist_url').to_dict()
def url_to_artist(url):
    return corresp['Name'][url]

df.Artiste1 = df.Artiste1.apply(url_to_artist)
df.Artiste2 = df.Artiste2.apply(url_to_artist)
df = df[['Artiste1', 'Artiste2']]

In [5]:
def is_northeast(city):
    if "connecticut" in city or 'maine' in city or 'massachusetts' in city or 'newhampshire' in city \
    or 'rhodeisland' in city or 'vermont' in city or 'newjersey' in city or 'newyork' in city or 'pennsylvania' in city: 
        return True
    
def is_midwest(city):
    if 'illinois'in city or 'indiana' in city or 'michigan' in city or 'ohio' in city or 'wisconsin' in city \
    or'iowa' in city or 'kansas' in city or 'minnesota' in city or 'missouri' in city or 'nebraska' in city \
    or 'northdakota' in city or 'southdakota' in city:
        return True
    
def is_south(city):
    if 'delaware' in city or 'florida' in city or 'georgia' in city or 'maryland' in city or 'northcarolina' in city \
    or 'southcarolina' in city or 'virginia' in city or 'columbia' in city or 'westvirginia' in city \
    or 'alabama' in city or 'kentucky' in city or 'mississippi' in city or 'tennessee' in city \
    or 'arkansas' in city or 'louisiana' in city or 'oklahoma' in city or 'texas' in city:
        return True
    
def is_west(city):
    if 'arizona' in city or 'colorado' in city or 'idaho' in city or 'montana' in city or 'nevada' in city \
    or' newmexico' in city or 'utah' in city or 'wyoming' in city or 'alaska' in city or 'california' in city \
    or 'hawaii' in city or 'oregon' in city or 'washington' in city:
        return True
    

In [6]:
def clean_city(city, city2):
    try:
        city = city.lower().replace('_','')
    except AttributeError:
        city = ''
    try:
        city2 = city2.lower().replace('_','')
    except AttributeError:
        city2 = ''

    if 'newyork' in city2:
        return 'New-York-City'
    elif 'chicago' in city2 or 'illinois' in city2 or 'indiana' in city2 or 'saintlouis' in city2 or 'michigan' in city2:
        return 'Chicago'
    elif 'orleans' in city2 or 'louisiana' in city2:
        return 'New-Orleans'
    elif 'newyork' in city or 'newyork' in city :
        return 'New-York-City'
    elif 'chicago' in city or 'illinois' in city or 'indiana' in city or 'saintlouis' in city or 'michigan' in city:
        return 'Chicago'
    elif 'orleans' in city2 or 'louisiana' in city2:
        return 'New-Orleans'
    
    elif is_northeast(city2):
        return 'NorthEast'
    elif is_midwest(city2):
        return 'MidWest'
    elif is_south(city2):
        return 'South'
    elif is_west(city2):
        return 'West'

    elif is_northeast(city):
        return 'NorthEast'
    elif is_midwest(city):
        return 'MidWest'
    elif is_south(city):
        return 'South'
    elif is_west(city):
        return 'West'

In [7]:
metadata["Area"] = metadata[['Birth Place','Death Place']].apply(lambda x: clean_city(*x), axis=1)

In [8]:
from ast import literal_eval
def fix_list(l):
    try:
        return literal_eval(l)
    except ValueError:
        return None
metadata.Genre = metadata.Genre.apply(fix_list)
metadata.Instruments = metadata.Instruments.apply(fix_list)

In [9]:
#Get main style

unic_genre = []
for genres in metadata.Genre:
    try:
        for i in range(len(genres)):
            unic_genre.append(genres[i])
    except:
        pass
unic_genre = Counter(unic_genre)
unic_genre = sorted(unic_genre.items(), key=lambda pair: pair[1], reverse=False)

def unic_style(styles):
    try:
        genres = styles.copy()
        if len(genres) == 1:
            return genres[0]
        else:
            if 'Jazz' in genres:
                genres.remove('Jazz')
            for i in range(len(unic_genre)):
                if unic_genre[i][0] in genres and len(genres)>1:
                    genres.remove(unic_genre[i][0])
            return genres[0]
    except (ValueError, TypeError, AttributeError):
        return None
    
metadata['Main_style'] = metadata.Genre.apply(unic_style)

In [10]:
def clean_instrument(instrument):
    temp = instrument.lower()
    if 'sax' in temp:
        return 'Saxophone'
    if 'pian' in temp:
        return 'Piano'
    if 'trump' in temp:
        return 'Trumpet'
    if 'tromb' in temp:
        return 'Trombone'
    if 'drum' in temp or 'percu' in temp:
        return 'Drums'
    if temp == 'human_voice' or 'sing' in temp or 'vocals' in temp:
        return 'Singer'
    if 'violin' in temp:
        return 'Violin'
    if 'clarinet' in temp:
        return 'Clarinet'
    if 'guitar' in temp or 'fender' in temp or 'gibson' in temp:
        return 'Guitar'
    if 'bass' in temp:
        return 'Double Bass'
    elif 'bass' in temp:
        return 'Bass'
    else:
        return temp

In [11]:
# Get main instrument

unic_instruments = []
for inst in metadata.Instruments:
    try:
        for i in range(len(inst)):
            unic_instruments.append(inst[i])
    except:
            pass
unic_instruments = Counter(unic_instruments)
unic_instruments = sorted(unic_instruments.items(), key=lambda pair: pair[1], reverse=False)

def unic_style(inst):
    try:
        instr = inst.copy()
        if len(instr) == 1:
            temp = instr[0]
        else:
            if 'Jazz' in instr:
                instr.remove('Jazz')
            for i in range(len(unic_instruments)):
                if unic_instruments[i][0] in instr and len(instr)>1:
                    instr.remove(unic_instruments[i][0])
            temp = instr[0]
        return clean_instrument(temp)
        
    except (IndexError, ValueError, TypeError, AttributeError):
        return None
    
metadata['Main_instrument'] = metadata.Instruments.apply(unic_style)

In [12]:
top_instruments = pd.crosstab(metadata.Main_instrument, columns='count').sort_values(by='count', ascending=False).head(15)['count'].index

In [13]:
def keep_top_instruments(i):
    if i in top_instruments:
        return i
    else:
        return 'Other'
metadata.Main_instrument = metadata.Main_instrument.apply(keep_top_instruments)

In [14]:
def birth_year_to_decade(year):
    try:
        return str(round(int(year), -1))+'s'
    except ValueError:
        return None
metadata["Birth_decade"] = metadata["Birth Year"].apply(birth_year_to_decade)

In [15]:
df.to_csv('data/clean_triples.csv')
metadata[["Name", "Area", "Main_style", "Main_instrument", "Birth_decade"]].to_csv("data/metadata_dataframe_clean.csv")

In [ ]:
adjacency_matrix = pd.crosstab(df.Artiste1, df.Artiste2)
idx = adjacency_matrix.columns.union(adjacency_matrix.index)
adjacency_matrix = adjacency_matrix.reindex(index = idx, columns=idx, fill_value=0).as_matrix()

In [ ]:
(adjacency_matrix == 0).sum().sum()/np.prod(adjacency_matrix.shape)

In [ ]:
adjacency_matrix.sum()

In [ ]:
G = nx.from_numpy_matrix(adjacency_matrix)

In [ ]:
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
# print "Degree sequence", degree_sequence
dmax = max(degree_sequence)

plt.plot(degree_sequence, 'b-', marker='o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")

In [ ]:
nx.closeness_centrality(G).values()

In [ ]:
def keep_row(lien):
    if 'collaborated_with' in lien or 'inBand' in lien or  'bandmember' in lien or "playedTogether" in lien :
        return True
    else:
        return False
df["Keep"] = df.Lien.apply(keep_row)
df = df[df.Keep == True]
df["Id"] = df.Artiste1+df.Artiste2
test = df[["Artiste1", 'Artiste2', "Id"]].drop_duplicates(subset='Id')